# Setup Apache Beam

In [ ]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 14.9 MB/s eta 0:00:00
  

# Basic Pipeline

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

# Define a basic pipeline
def run_basic_pipeline():
    # Create a pipeline using DirectRunner (local execution)
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        # Simple pipeline that multiplies each number by 10
        numbers = (pipeline
                   | 'Create Numbers' >> beam.Create([1, 2, 3, 4, 5])
                   | 'Multiply by 10' >> beam.Map(lambda x: x * 10)
                   | 'Print Output' >> beam.Map(print))

# Run the basic pipeline
run_basic_pipeline()


10
20
30
40
50


# Composite Transforms

In [ ]:
# Composite transform to multiply numbers by 10 and add 5
class MultiplyAndAdd(beam.PTransform):
    def expand(self, pcoll):
        return (pcoll
                | 'Multiply by 10' >> beam.Map(lambda x: x * 10)
                | 'Add 5' >> beam.Map(lambda x: x + 5))

def run_composite_transform_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (pipeline
         | 'Create Numbers' >> beam.Create([1, 2, 3, 4, 5])
         | 'Apply Composite Transform' >> MultiplyAndAdd()
         | 'Print Output' >> beam.Map(print))

run_composite_transform_pipeline()


15
25
35
45
55


# Pipeline I/O (Reading and Writing Data)

In [ ]:
def run_pipeline_io():
    # Reading from a text file and writing to another text file
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (pipeline
         | 'Read from File' >> beam.io.ReadFromText('input.txt')
         | 'Convert to Uppercase' >> beam.Map(str.upper)
         | 'Write to File' >> beam.io.WriteToText('output'))

# Create a simple input file
with open('input.txt', 'w') as f:
    f.write("hello\nworld\napache\nbeam")

run_pipeline_io()

# Output file will be written in the 'output-00000-of-00001' file.
!cat output-00000-of-00001


HELLO
WORLD
APACHE
BEAM


# ParDo Transform

In [ ]:
class MultiplyDoFn(beam.DoFn):
    def process(self, element):
        yield element * 10

def run_pardo_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (pipeline
         | 'Create Numbers' >> beam.Create([1, 2, 3, 4, 5])
         | 'Multiply using ParDo' >> beam.ParDo(MultiplyDoFn())
         | 'Print Output' >> beam.Map(print))

run_pardo_pipeline()


10
20
30
40
50


# Windowing

In [ ]:
import time
import random

def run_windowing_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (pipeline
         | 'Create Numbers with Timestamps' >> beam.Create([(i, random.randint(1, 100)) for i in range(10)])
         | 'Add Timestamps' >> beam.Map(lambda x: beam.window.TimestampedValue(x, time.time()))
         | 'Window into Fixed Intervals' >> beam.WindowInto(beam.window.FixedWindows(10))
         | 'Sum per Window' >> beam.CombinePerKey(sum)
         | 'Print Output' >> beam.Map(print))

run_windowing_pipeline()


(0, 1)
(1, 39)
(2, 63)
(3, 47)
(4, 97)
(5, 71)
(6, 60)
(7, 22)
(8, 71)
(9, 76)


# Triggers

In [ ]:
from apache_beam.transforms.trigger import AfterWatermark, AccumulationMode
import time
import random
from apache_beam import window

def run_trigger_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (pipeline
         | 'Create Stream' >> beam.Create([(i, random.randint(1, 100)) for i in range(10)])
         | 'Add Timestamps' >> beam.Map(lambda x: beam.window.TimestampedValue(x, time.time()))
         | 'Window into Fixed Intervals' >> beam.WindowInto(
             beam.window.FixedWindows(5),
             trigger=AfterWatermark(),  # Use AfterWatermark trigger
             accumulation_mode=AccumulationMode.DISCARDING)
         | 'Sum per Window with Trigger' >> beam.CombinePerKey(sum)
         | 'Print Output' >> beam.Map(print))

run_trigger_pipeline()


(0, 13)
(1, 13)
(2, 53)
(3, 79)
(4, 28)
(5, 63)
(6, 24)
(7, 64)
(8, 72)
(9, 2)


# Streaming Pipeline

In [ ]:
import time

def run_streaming_pipeline():
    with beam.Pipeline(options=PipelineOptions(streaming=True)) as pipeline:
        (pipeline
         | 'Generate Sequence' >> beam.io.ReadFromPubSub(topic='projects/my-project/topics/my-topic')
         | 'Add 5 to each element' >> beam.Map(lambda x: int(x) + 5)
         | 'Print Output' >> beam.Map(print))

# Note: The streaming example requires a real Pub/Sub source for true streaming.
